In [10]:
from npodjlEnv import NpodJLEnv
import numpy as np
env = NpodJLEnv()
env.ver()

0.4

In [11]:
def argmaxQ(Q, s, n_actions):
    if s in Q.keys():
        print("Policy: Greedy")
        return(np.argmax(Q[s]))
    else:
        print("Policy: Random")
        return(np.random.randint(0, n_actions))

In [12]:
def policy_greedy(Q, s, epsilon, n_actions):
    if np.random.rand() < epsilon:
        return(argmaxQ(Q,s,n_actions))
    else:
        print("Policy: Random")
        return(np.random.randint(0, n_actions))

In [13]:
def SARSA(env, alpha, gamma, epsilon, Q, max_cylces):
    n_actions = len(env.actions())
    r = [None] * (2*max_cycles)
    s = [None] * (2*(max_cycles+1))
    a = [None] * (2*max_cycles)
    
    t=0
    s[t] = env.encoded_state()
    a[t] = policy_greedy(Q,s[t], epsilon, n_actions)
    obs = env.run(a[t])
    r[t] = obs['reward']
    s[t+1] = env.encoded_state() 
    if not s[t] in Q.keys():
        Q[s[t]] = [0]*n_actions
    try:
        while t < max_cycles:
            print("State: ",env.encoded_state())
            a[t+1] = policy_greedy(Q,s[t+1], epsilon, n_actions)
            print("Action: ", a[t])
            print("-----")
            obs = env.run(a[t])
            r[t+1] = obs['reward']
            s[t+2] = env.encoded_state()
            if not s[t+1] in Q.keys():
                Q[s[t+1]] = [0]*n_actions
            Q[s[t]][a[t]] = Q[s[t]][a[t]] + alpha * ( r[t] + (gamma * Q[s[t+1]][a[t+1]] ) - Q[s[t]][a[t]] )
            t += 1
            
    except KeyboardInterrupt:
        print("Press Ctrl-C to terminate while statement")
        pass
    return(s,a,r,Q)

In [5]:
Q = dict()

In [7]:
env.reset()
alpha = 1
gamma = 0.99
epsilon = 0.5
max_cycles = 20
#sol = SARSA(env, alpha, gamma, epsilon, Q, max_cycles)

In [9]:
Q

{'0.001000_50.000000_2.000000_250.000000_0.000000_0.100000': [-645.15000218514,
  -483.64987936809763,
  -387.0803675533716,
  -322.4304338430341,
  -361.35960254028697,
  -241.3967792951297,
  -214.9564683361284,
  -829.1063552790074,
  0,
  -1934.632251923576,
  0,
  0],
 '0.001000_50.000000_2.000000_227.272727_0.000000_0.100000': [0,
  -34.66126853180549,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 '0.001000_50.000000_2.000000_206.611570_0.000000_0.100000': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1.705804107394897],
 '0.001000_50.000000_2.200000_206.611570_0.000000_0.100000': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  -20.22474830803435,
  0,
  0,
  0,
  0],
 '0.001000_50.000000_2.200000_206.611570_0.000000_0.090909': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  -2.0008123097500174,
  0],
 '0.001000_50.000000_2.000000_206.611570_0.000000_0.090909': [0.0,
  0,
  0,
  0,
  0.12608674695889022,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 '0.001100_50.000000_2.000000_206

In [14]:
for i in range(1,10):
        print(i)
        env.reset()
        SARSA(env, 1/i, 0.9, 1 - 1/i, Q, 20)

1
State:  0.001000_50.000000_2.000000_227.272727_0.000000_0.100000
Action:  9
-----
State:  0.001000_50.000000_2.000000_206.611570_0.000000_0.100000
Action:  1
-----
State:  0.001000_50.000000_2.200000_206.611570_0.000000_0.100000
Action:  6
-----
State:  0.000909_50.000000_2.200000_206.611570_0.000000_0.100000
Action:  8
-----
State:  0.000909_45.454545_2.200000_206.611570_0.000000_0.100000
Action:  1
-----
State:  0.000909_45.454545_2.420000_206.611570_0.000000_0.100000
Action:  8
-----
State:  0.000909_41.322314_2.420000_206.611570_0.000000_0.100000
Action:  5
-----
State:  0.000909_41.322314_2.420000_206.611570_0.000000_0.110000
Action:  1
-----
State:  0.000909_41.322314_2.662000_206.611570_0.000000_0.110000
Action:  9
-----
State:  0.000909_41.322314_2.662000_187.828700_0.000000_0.110000
Action:  0
-----
State:  0.001000_41.322314_2.662000_187.828700_0.000000_0.110000
Action:  9
-----
State:  0.001000_41.322314_2.662000_170.753364_0.000000_0.110000
Action:  11
-----
State:  0.001

6
State:  0.000909_50.000000_2.000000_250.000000_0.000000_0.100000
Action:  6
-----
State:  0.000826_50.000000_2.000000_250.000000_0.000000_0.100000
Action:  0
-----
State:  0.000909_50.000000_2.000000_250.000000_0.000000_0.100000
Action:  4
-----
State:  0.000909_50.000000_2.000000_250.000000_0.000000_0.100000
Action:  1
-----
State:  0.000909_50.000000_2.200000_250.000000_0.000000_0.100000
Action:  1
-----
State:  0.000909_50.000000_2.420000_250.000000_0.000000_0.100000
Action:  6
-----
State:  0.000826_50.000000_2.420000_250.000000_0.000000_0.100000
Action:  6
-----
State:  0.000751_50.000000_2.420000_250.000000_0.000000_0.100000
Action:  8
-----
State:  0.000751_45.454545_2.420000_250.000000_0.000000_0.100000
Action:  0
-----
State:  0.000826_45.454545_2.420000_250.000000_0.000000_0.100000
Action:  4
-----
State:  0.000826_45.454545_2.420000_250.000000_0.000000_0.100000
Action:  10
-----
State:  0.000826_45.454545_2.420000_250.000000_0.000000_0.100000
Action:  0
-----
State:  0.000

RuntimeError: <PyCall.jlwrap (in a Julia function called from Python)
JULIA: DimensionMismatch("second dimension of A, 43, does not match length of x, 42")
Stacktrace:
 [1] gemv!(::Array{Float64,1}, ::Char, ::Array{Float64,2}, ::Array{Float64,1}, ::Bool, ::Bool) at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.3/LinearAlgebra/src/matmul.jl:451
 [2] mul! at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.3/LinearAlgebra/src/matmul.jl:66 [inlined]
 [3] mul! at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.3/LinearAlgebra/src/matmul.jl:203 [inlined]
 [4] * at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.3/LinearAlgebra/src/matmul.jl:47 [inlined]
 [5] Dopt(::LinearAlgebra.Adjoint{Float64,Array{Float64,2}}, ::Array{Float64,2}, ::Array{Float64,2}, ::Float64, ::Float64, ::Array{Float64,1}, ::Array{Float64,1}) at /Users/julianotalvaro/.julia/packages/npod/HCWf1/src/npod.jl